## Greedy Algorithm

In [30]:
import os, sys
sys.path.append("..")

In [31]:
from opttrot.pauli import PauliElement
from opttrot.pauli_utils import sym_code2pstr

import numpy as np

In [32]:
N = 3

In [33]:
p1 = PauliElement(2, 2, N, 1)

In [34]:
parr = np.array([PauliElement(0, 0, N, 1), PauliElement(1, 0, N, 1), PauliElement(0, 1, N, 1)])
parr

array([PauliElement(n=3, weight=1.000000+(0.000000)j, III),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IIX),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IIZ)], dtype=object)

In [35]:
parr[0] = p1@parr[0]

In [36]:
parr[2] = p1@parr[2]

In [37]:
parr

array([PauliElement(n=3, weight=1.000000+(0.000000)j, IYI),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IIX),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IYZ)], dtype=object)

In [90]:
class PauliFrame:
    def __init__(self, n):
        self.qubit = n

        self.front = np.vstack([np.zeros(self.qubit), 2**np.arange(self.qubit)]).T.astype(int)
        self.back  = np.vstack([2**np.arange(self.qubit), np.zeros(self.qubit)]).T.astype(int)
        self.sign = np.ones(self.qubit, dtype=int)
    def __repr__(self):
        reps = []
        for i, (x1, z1, x2, z2)  in enumerate(np.hstack([self.front, self.back])):
            reps.append(f"{self.sign[i]}, {sym_code2pstr((x1, z1),self.qubit)}, {sym_code2pstr((x2, z2), self.qubit)}\n")
        return "".join(reps)
    
    def h(self, i):
        self.back[i][0], self.front[i][0] = self.front[i][0], self.back[i][0]
        self.back[i][1], self.front[i][1] = self.front[i][1], self.back[i][1]
    def s(self, i):
        pass
    def s_d(self, i):
        pass 
    def two_entangle(self, i, j, c_type = 0):
        # 0 = cx
        # 1 = cz
        # 2 = cy

In [91]:
pf = PauliFrame(5)
pf

1, IIIIZ, IIIIX
1, IIIZI, IIIXI
1, IIZII, IIXII
1, IZIII, IXIII
1, ZIIII, XIIII

In [93]:
pf.h(3)
pf

1, IIIIZ, IIIIX
1, IIIZI, IIIXI
1, IIZII, IIXII
1, IXIII, IZIII
1, ZIIII, XIIII

In [89]:
pf.front, pf.back

(array([[ 0,  1],
        [ 0,  2],
        [ 0,  4],
        [ 8,  0],
        [ 0, 16]]),
 array([[ 1,  0],
        [ 2,  0],
        [ 4,  0],
        [ 0,  8],
        [16,  0]]))

In [64]:
np.arange(4)

array([0, 1, 2, 3])

In [21]:
np.vstack([2**np.arange(4), np.zeros(4)]).T

array([[1., 0.],
       [2., 0.],
       [4., 0.],
       [8., 0.]])

In [23]:
h = np.vstack([2**np.arange(4), np.zeros(4)]).T

In [26]:
np.hstack([h, h])

array([[1., 0., 1., 0.],
       [2., 0., 2., 0.],
       [4., 0., 4., 0.],
       [8., 0., 8., 0.]])